### Todo:
- Get all possible permutations of a replacement schedule (with 1 replacement, 2, 3)
- make replacement schedules flexible with more than 15 year timelines (i.e. more than 3 replacements)
- ~~vehicle age matrix~~
- ~~vehicle_mileage matrix
- ~~odometer matrix
- ~~acquisition_cost matrix
    - (pct change factor)
- ~~consumables_cost matrix
- ~~maintenance_cost matrix
- Do a whole round of scrutiny to make this code much more flexible. 
- ~~figure out how to flag infeasible schedules
- add number of charging stations to build and number of charging stations to operate as vars
- make toy example with two vehicles
- add penalty for going over budget or under emissions

In [1]:
import numpy as np
from sympy.utilities.iterables import multiset_permutations

np.set_printoptions(edgeitems=15,linewidth=600)
# np.core.arrayprint._line_width = 400

In [2]:
import time
start = time.time()

In [3]:
numVehicles = 20

### Schedules 

In [4]:
#todo: will likely need a generalizable method for computing the maximum number of replacements possible
oneReplacement = np.array([1,0,0,0,0,0,0,0,0,0,0,0,0,0,0])
twoReplacements = np.array([1,1,0,0,0,0,0,0,0,0,0,0,0,0,0])
threeReplacements = np.array([1,1,1,0,0,0,0,0,0,0,0,0,0,0,0])

oneReplacement = list(multiset_permutations(oneReplacement))
twoReplacements = list(multiset_permutations(twoReplacements))
threeReplacements = list(multiset_permutations(threeReplacements))

replacementSchedules = np.array(oneReplacement+twoReplacements+threeReplacements)
# replacementSchedules = np.array(twoReplacements+threeReplacements)

In [5]:
replacementSchedules.shape

(575, 15)

In [6]:
numSchedules,_ = replacementSchedules.shape

In [7]:
replacementSchedules = np.repeat(replacementSchedules[np.newaxis,:, :], numVehicles, axis=0)

In [8]:
#opposite of replacement -- may want to use this at some point
keepSchedules = (replacementSchedules-1)*-1
# keepSchedules#[50:80]

### Age

In [9]:
def initialize_vehicle_age(keepSchedules,startingAge=0):
    """Gets the vehicle age building process started. Produces a matrix for the age of the vehicle according to the replacement schedule, which is later fixed by get_vehicle_age"""
    age = startingAge+np.cumsum(keepSchedules,axis=2)*keepSchedules
    age[age==startingAge] = 0 #fixes the fact that replaced vehicles start at 0 (if this wasn't here they would start at the starting age)
    return age


def get_vehicle_age(age=None,k=1,startingAge=0):
    if k==1:
        age = initialize_vehicle_age(keepSchedules,startingAge)

    diff = np.diff(age,axis=2)
    diffMask = np.append(np.ones(shape=(numVehicles,numSchedules,1)),diff,axis=2)>1
    age[diffMask]=k

    if age[diffMask].size==0:
        return age
    else:
        return get_vehicle_age(age,k=k+1)

age = get_vehicle_age(startingAge=0)#[50:80]

### Mileage

In [10]:
#vehicle_mileage matrix 
# vehicle_mileage = np.repeat(np.round(np.random.normal(loc=10000,scale=2000,size=(1, 15))),numSchedules,axis=0)
annual_mileage = np.ones(shape=(numVehicles,numSchedules,15))
annual_mileage[0,:,:]*=np.round(np.random.normal(loc=10000,scale=2000))
annual_mileage[1,:,:]*=np.round(np.random.normal(loc=11000,scale=2000))

In [11]:
#matrix showing what the odometer reading will be under each schedule
odometer = annual_mileage*age

### Acquisition

In [12]:
np.random.randint(20000,40000,size=(numVehicles))

array([34261, 31582, 39663, 34367, 39524, 28894, 29868, 22848, 36648, 36257, 33696, 21784, 21266, 29988, 20692, 36831, 22317, 35696, 35102, 27006])

In [13]:
#Todo: need to look up acquisitoin cost for each vehicle
def get_acquisition_cost(replacementSchedules):
    acquisition = replacementSchedules.copy()
    for v in range(numVehicles):
        acquisition[v,:,:]*=np.random.randint(20000,40000)
    return acquisition

acquisition = get_acquisition_cost(replacementSchedules)

### Consumables

In [14]:
firstReplacements = np.argmax(replacementSchedules[0]==1,axis=1)
# firstReplacements

In [15]:
def get_vehicle_type_trackers(replacementSchedules):
    """Returns two matrices. One that tracks if in a givemn year a schedule implies the vehicle is still an ice, and then the opposite: whether or not in a given year a vehicle is now an EV"""
    firstReplacements = np.argmax(replacementSchedules[0]==1,axis=1) #gets index of year the vehicle is first replaced (ie it transitions from ICE to EV)
    is_ice = replacementSchedules.copy()
    is_ev = replacementSchedules.copy()
    
    for i in range(0,numSchedules): #there is most definitely a better way to do this in numpy but I took way too long researching
        is_ice[:,i,firstReplacements[i]:] = 0
        is_ice[:,i,:firstReplacements[i]] = 1

        is_ev[:,i,firstReplacements[i]:] = 1
        is_ev[:,i,:firstReplacements[i]] = 0
    return is_ice,is_ev

is_ice,is_ev = get_vehicle_type_trackers(replacementSchedules)

In [16]:
def get_consumables(is_ice,is_ev):
    """Will make this function more flexible later. Calculates fuel cost as if always ICE and always EV. And then applies to the schedules based on when the initial transition from ICE to EV occurs. """   
    #ICE
    #fuel $ = mileage/mpg*cpg
    cpg = 2.25
    mpg = 22
    fuel = annual_mileage/mpg*cpg

        #EV
    #fuel $ = mileage/mpeg * cpeg
    cpeg = 1.2
    mpeg = 100
    electricity = annual_mileage/mpeg*cpeg

    consumables = np.round(fuel*is_ice+(electricity*is_ev))
    return consumables

consumables = get_consumables(is_ice,is_ev)

### Maintenance cost

In [17]:
def get_maintenance_cost(age,annual_mileage,odometer):
    """- ! because this is likely to change. For now I'm just going to treat as a linear regression with made up coeffs."""
    age_coef = .01
    mileage_coef = .2
    odometer_coef = .1
    maintenance = (age_coef*age)+(mileage_coef*annual_mileage)+(odometer_coef*odometer)
    return maintenance

maintenance = get_maintenance_cost(age,annual_mileage,odometer)

### Emissons

In [18]:
def get_emissions(is_ice,is_ev):
    """Will make this function more flexible later. Calculates fuel cost as if always ICE and always EV. And then applies to the schedules based on when the initial transition from ICE to EV occurs. """   
    #calc: kg CO2/gallon * mileage/mpg
    
    ice_emission_factor = 2.421
    mpg = 22
    ice_emissions = ice_emission_factor*annual_mileage/mpg

    ev_emission_factor = 0
    mpge = 100
    ev_emissions = ev_emission_factor*annual_mileage/mpge

    emissions = np.round((ice_emissions*is_ice)+(ev_emissions*is_ev))
    return emissions

emissions = get_emissions(is_ice,is_ev)

### Find and filter out infeasible schedules
- infeasible in the sense that they have a replacement happening in years where the vehicle is both under 6 years old and doesn't yet have 150k miles on it

In [19]:
odometer_diff = np.diff(odometer)
odometer_check = (odometer_diff>-150000) & (odometer_diff<=0)

In [20]:
age_diff = np.diff(age)
age_check = (age_diff>-6) & (age_diff<=0)#.any()


In [21]:
both_check = odometer_check*age_check

In [22]:
infeasible_filter = both_check.any(axis=2)

In [23]:
# replacementSchedules[~infeasible_filter]

In [24]:
#!I feel like there should be more feasible schedules. Will need to come back to this. 
def find_infeasible_schedules(odometer,age):
    """Generates a mask that is True for any schedule that is infeasible. These can be filtered out before running the model."""
    odometer_diff = np.diff(odometer)
    odometer_check = (odometer_diff>-150000) & (odometer_diff<=0)

    age_diff = np.diff(age)
    age_check = (age_diff>-6) & (age_diff<=0)#.any()

    both_check = odometer_check*age_check
    is_infeasible = both_check.any(axis=2)
    return is_infeasible

infeasible_filter = find_infeasible_schedules(odometer,age)
# replacementSchedules[~infeasible_filter]

### Toy Model 

In [25]:
import gurobipy as grb

In [26]:
num_vehicles,num_schedules,num_years = replacementSchedules.shape

In [27]:
vehicles = [v for v in range(0,num_vehicles)]
schedules = [s for s in range(0,num_schedules)]
years = [t for t in range(0,num_years)]
finalYear = max(years)

In [28]:
#! I think I can pull the code I used in modelinputsgen
budget_acquisition = 1300000*np.ones(shape=(15)) 
budget_operations = 1000000*np.ones(shape=(15))

emissions_goal = 40000

numDesiredSolutions = 3

In [29]:
try: 
    m.reset()
    del m    
except:
    None
    
m = grb.Model('carnet')


--------------------------------------------
--------------------------------------------

Using license file C:\Users\elynch\gurobi.lic
Academic license - for non-commercial use only


In [30]:
m.setParam('PoolSearchMode',2) #tell gurobi I want multiple solutions
m.setParam('PoolSolutions',numDesiredSolutions) #number of solutions I want

Changed value of parameter PoolSearchMode to 2
   Prev: 0  Min: 0  Max: 2  Default: 0
Changed value of parameter PoolSolutions to 3
   Prev: 10  Min: 1  Max: 2000000000  Default: 10


In [31]:
x = m.addVars(vehicles,schedules,vtype=grb.GRB.BINARY,name='x')
# y = m.addVars(years,)

In [32]:
w = {'cost':0.01,'emissions':0.99}

In [33]:
#todo: filter out infeasible schedules before running the model. Going to take too long without doing this. 

In [34]:
total_cost = np.sum(consumables+acquisition+maintenance,axis=2)

In [35]:
obj = m.setObjective(grb.quicksum(w['cost']*total_cost[v,s]*x[v,s] for v in vehicles for s in schedules) + 
                                  grb.quicksum(w['emissions']*emissions[v,s,finalYear]*x[v,s] for v in vehicles for s in schedules),grb.GRB.MINIMIZE)
# obj = m.setObjective(grb.quicksum(w['cost']*(consumables[v,s,t]+acquisition[v,s,t]+maintenance[v,s,t])*x[v,s] + 
#                                   w['emissions']*emissions[v,s,finalYear]*x[v,s] for v in vehicles for s in schedules for t in years),grb.GRB.MINIMIZE)

Wall time: 0 ns


In [36]:
c1 = m.addConstrs((grb.quicksum(x[v,s] for s in schedules)==1 for v in vehicles),'one_schedule_per_vehicle')

In [37]:
c2 = m.addConstrs((grb.quicksum((consumables[v,s,t]+maintenance[v,s,t])*x[v,s] for v in vehicles for s in schedules) <= budget_operations[t] for t in years),'operations_budget')
c2 = m.addConstrs((grb.quicksum(acquisition[v,s,t]*x[v,s] for v in vehicles for s in schedules) <= budget_acquisition[t] for t in years),'acquisition_budget')

In [38]:
c4 = m.addConstr(grb.quicksum(emissions[v,s,finalYear]*x[v,s] for v in vehicles for s in schedules) <= emissions_goal)

In [39]:
c5 = m.addConstrs((infeasible_filter[v,s]*x[v,s] <= 0 for v in vehicles for s in schedules),'infeasible_schedules')

In [40]:
m.optimize()

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 11551 rows, 11500 columns and 226860 nonzeros
Model fingerprint: 0x796a89f7
Variable types: 0 continuous, 11500 integer (11500 binary)
Coefficient statistics:
  Matrix range     [2e-01, 4e+04]
  Objective range  [2e+02, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+06]
Found heuristic solution: objective 10976.291900
Presolve removed 11531 rows and 11060 columns
Presolve time: 0.02s
Presolved: 20 rows, 440 columns, 440 nonzeros
Variable types: 0 continuous, 440 integer (440 binary)

Root relaxation: objective 7.838048e+03, 20 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    7838.0480000 7838.04800  0.00%     -    0s

Optimal solution found at node 0 - now completing solution pool...

    Nodes    |    Current Node    |      Pool

In [41]:
end = time.time()
print("--- %s seconds ---" % (time.time() - start))


--- 13.10148572921753 seconds ---


In [42]:
#multiple solutions
solution = []
options = ['A','B','C']
for solution in range(0,3):
    print()
    print(f'Option: {options[solution]}')
    m.setParam('SolutionNumber',solution)
    for v in vehicles:
        for s in schedules:
            if x[v,s].xn==1:
                print(f'   Vehicle: {v+1} Schedule: {s} {replacementSchedules[v,s]}')     


Option: A
   Vehicle: 1 Schedule: 7 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
   Vehicle: 2 Schedule: 7 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
   Vehicle: 3 Schedule: 7 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
   Vehicle: 4 Schedule: 7 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
   Vehicle: 5 Schedule: 7 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
   Vehicle: 6 Schedule: 7 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
   Vehicle: 7 Schedule: 7 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
   Vehicle: 8 Schedule: 7 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
   Vehicle: 9 Schedule: 7 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
   Vehicle: 10 Schedule: 7 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
   Vehicle: 11 Schedule: 7 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
   Vehicle: 12 Schedule: 7 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
   Vehicle: 13 Schedule: 7 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
   Vehicle: 14 Schedule: 7 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
   Vehicle: 15 Schedule: 7 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
   Vehicle: 16 Schedule: 7 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
   Vehicle: 17 Schedule: 7 [0 0 0 0 0 0 0 1 0 0 0 0 0 

In [43]:
print(f'age: {age[0,29]}')
print()
print(f'annual_mileage: {annual_mileage[0,29]}')
print()
print(f'odometer: {odometer[0,29]}')
print()
print(f'acquisition cost: {acquisition[0,29]}')
print()
print(f'maintenance cost: {maintenance[0,29]}')
print()
print(f'consumables cost: {consumables[0,29]}')
print()
print(f'emissions: {emissions[0,29]}')

age: [1 2 3 4 5 6 7 8 9 0 0 1 2 3 4]

annual_mileage: [11397. 11397. 11397. 11397. 11397. 11397. 11397. 11397. 11397. 11397. 11397. 11397. 11397. 11397. 11397.]

odometer: [ 11397.  22794.  34191.  45588.  56985.  68382.  79779.  91176. 102573.      0.      0.  11397.  22794.  34191.  45588.]

acquisition cost: [    0     0     0     0     0     0     0     0     0 25602 25602     0     0     0     0]

maintenance cost: [ 3419.11  4558.82  5698.53  6838.24  7977.95  9117.66 10257.37 11397.08 12536.79  2279.4   2279.4   3419.11  4558.82  5698.53  6838.24]

consumables cost: [1166. 1166. 1166. 1166. 1166. 1166. 1166. 1166. 1166.  137.  137.  137.  137.  137.  137.]

emissions: [1254. 1254. 1254. 1254. 1254. 1254. 1254. 1254. 1254.    0.    0.    0.    0.    0.    0.]


In [44]:
len(m.x)

11500

In [45]:
emissions[7]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.

In [46]:
consumables[7]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.

In [47]:
maintenance[7]

array([[0.31, 0.42, 0.53, 0.64, 0.75, 0.86, 0.97, 1.08, 1.19, 1.3 , 1.41, 1.52, 1.63, 1.74, 0.2 ],
       [0.31, 0.42, 0.53, 0.64, 0.75, 0.86, 0.97, 1.08, 1.19, 1.3 , 1.41, 1.52, 1.63, 0.2 , 0.31],
       [0.31, 0.42, 0.53, 0.64, 0.75, 0.86, 0.97, 1.08, 1.19, 1.3 , 1.41, 1.52, 0.2 , 0.31, 0.42],
       [0.31, 0.42, 0.53, 0.64, 0.75, 0.86, 0.97, 1.08, 1.19, 1.3 , 1.41, 0.2 , 0.31, 0.42, 0.53],
       [0.31, 0.42, 0.53, 0.64, 0.75, 0.86, 0.97, 1.08, 1.19, 1.3 , 0.2 , 0.31, 0.42, 0.53, 0.64],
       [0.31, 0.42, 0.53, 0.64, 0.75, 0.86, 0.97, 1.08, 1.19, 0.2 , 0.31, 0.42, 0.53, 0.64, 0.75],
       [0.31, 0.42, 0.53, 0.64, 0.75, 0.86, 0.97, 1.08, 0.2 , 0.31, 0.42, 0.53, 0.64, 0.75, 0.86],
       [0.31, 0.42, 0.53, 0.64, 0.75, 0.86, 0.97, 0.2 , 0.31, 0.42, 0.53, 0.64, 0.75, 0.86, 0.97],
       [0.31, 0.42, 0.53, 0.64, 0.75, 0.86, 0.2 , 0.31, 0.42, 0.53, 0.64, 0.75, 0.86, 0.97, 1.08],
       [0.31, 0.42, 0.53, 0.64, 0.75, 0.2 , 0.31, 0.42, 0.53, 0.64, 0.75, 0.86, 0.97, 1.08, 1.19],
       [0.

### Old stuff from vehicle age recursion 